In [1]:
import sys
sys.path.extend(['/Users/philipmassey/PycharmProjects/stock_market'])

import portfolio as pf
import market_data as md
import pandas as pd
from datetime import datetime
import pymongo
from pymongo import MongoClient
client = MongoClient()
db = client['stock_market']
db.list_collection_names()

['market_data_close', 'market_data_volume']

In [2]:
#db['market_data_close'].drop()
# db['market_data_volume'].drop()b
# print(list(db.list_collection_names()))

In [64]:
def removeBadData(df):
    if 'BRK.B' in list(df.columns):
        print(df['BRK.B'])
        return df.drop(columns=['BRK.B'])
    return df

def addPicklesToMdbConn(ndays):
    print(ndays)
    dfall = md.getStockPickleNBDays(ndays,skip=True)
    if dfall.size > 0:
        df = dfall['Close']
        df = removeBadData(df)
        df.reset_index(inplace=True)
        data_dict = df.to_dict("records")
        db["market_data_close"].insert_many(data_dict)
        df = dfall['Volume']
        df = removeBadData(df)
        df.reset_index(inplace=True)
        data_dict = df.to_dict("records")
        db["market_data_volume"].insert_many(data_dict)

[addPicklesToMdbConn(ndays) for ndays in range(33,60)]


33
Date
2020-12-01   NaN
Name: BRK.B, dtype: float64
Date
2020-12-01   NaN
Name: BRK.B, dtype: float64
34
Date
2020-11-30   NaN
Name: BRK.B, dtype: float64
Date
2020-11-30   NaN
Name: BRK.B, dtype: float64
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [65]:
from bson import json_util, ObjectId
from pandas import json_normalize
import json
from datetime import datetime

def getMdbDateFromNdays(ndays):
    strDate = md.getNBusDateFromNdays(ndays)
    return datetime.strptime(strDate,'%Y-%m-%d')

def MdbToDataframe(mongo_data):
        sanitized = json.loads(json_util.dumps(mongo_data))
        normalized = json_normalize(sanitized)
        df = pd.DataFrame(normalized)
        df['Date'] = datetime.utcfromtimestamp(float(df['Date.$date']/1e3))
        df.set_index('Date',inplace=True)
        df.drop(['Date.$date','_id.$oid'], axis=1,inplace=True)
        return df

def getNdaysRowFromMdb(ndays,db_coll):
    adate = getMdbDateFromNdays(ndays)
    mdb_data = db_coll.find({'Date':adate})[0]
    return MdbToDataframe(mdb_data)

def getMissingMarketRow(ndays,db_coll,symbols):
    df = getNdaysRowFromMdb(ndays,db_col)
    dfs = set(df.columns)
    missing_symbols = set(symbols).difference(dfs)
    if len(missing_symbols) > 0:
        return md.getRowNDaysAgo(ndays,missing_symbols)
    else:
        return pd.DataFrame({})
    
def updateMdbWithRow(ndays,df_m,db_coll):
    if df_m.size > 1:
        df_m = df_m.dropna(axis='columns')
        df_m.reset_index(inplace=True)
        df_m.drop(columns=['Date'],inplace=True)
        data_dict = df_m.to_dict("records")
        #print(data_dict)
        newvalues = { "$set":data_dict[0]}
        dt = getMdbDateFromNdays(ndays)
        query = {'Date':dt}
        result = db_coll.update_one(query, newvalues)  
        print(ndays,len(data_dict[0]),result.matched_count, result.modified_count )

def updateRowWithMissing(ndays,db_coll,symbols):
    df_m = getMissingMarketRow(ndays,symbols)
    updateMdbWithRow(df_m,db_coll)
    
def updateMdbWithRow(ndays,symbols):
    print(ndays)
    db_coll = db["market_data_close"]
    df_m = getMissingMarketRow(ndays,db_coll,symbols)
    if df_m.size > 0:
        updateMdbWithRow(ndays,df_m['Close'],db_coll)
        db_coll = db["market_data_volume"]
        #df_m = getMissingMarketRow(ndays,df_col)
        updateMdbWithRow(ndays,df_m['Volume'],db_coll)


symbols = md.getPortfoliosSymbols()
#updateMdbWithMissingRow(7,symbols)
[updateMdbWithMissingRow(ndays,symbols) for ndays in range(31,60)]

31
[*********************100%***********************]  25 of 25 completed
31 24 1 1
31 24 1 1
32


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


[*********************100%***********************]  25 of 25 completed
32 24 1 1
32 24 1 1
33
[*********************100%***********************]  26 of 26 completed
33 25 1 1
33 25 1 1
34
[*********************100%***********************]  27 of 27 completed
34 26 1 1
34 26 1 1
35
[*********************100%***********************]  40 of 40 completed
35 39 1 1
35 39 1 1
36
[*********************100%***********************]  40 of 40 completed
36 39 1 1
36 39 1 1
37
[*********************100%***********************]  1 of 1 completed
38
[*********************100%***********************]  40 of 40 completed
38 39 1 1
38 39 1 1
39
[*********************100%***********************]  40 of 40 completed
39 39 1 1
39 39 1 1
40
[*********************100%***********************]  40 of 40 completed
40 39 1 1
40 39 1 1
41
[*********************100%***********************]  40 of 40 completed
41 39 1 1
41 39 1 1
42
[*********************100%***********************]  40 of 40 completed
42 38 1 1
4

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [24]:
df_missing

,Close,Volume
Date,,
2021-01-08,0.98,604200


In [26]:
#print('size',df_m.size)
updateRowWithMissing(df_missing['Close'],db["market_data_close"])
updateRowWithMissing(df_missing['Volume'],db["market_data_volume"])

[*********************100%***********************]  1 of 1 completed


In [73]:
print(ndays,len(data_dict[0]),result.matched_count, result.modified_count )

12 6 1 1


In [54]:
newvalues = { "$set": { 'DOG': 'poor boy','DOGGY':'hush now' } } 
def replaceNanVol(df):
    return df_m['Close'].dropna(axis='columns')
df_missing = df_m['Close']
df_missing = replaceNanVol(df_missing)
df_missing.reset_index(inplace=True)
df_missing.drop(columns=['Date'],inplace=True)
data_dict = df_missing.to_dict("records")
data_dict[0]



{'NXGB': 0.10509999841451645,
 'VREOF': 1.4795000553131104,
 'WUGI': 50.32600021362305}

In [57]:
newvalues = { "$set":data_dict[0]}
result = db["market_data_close"].update_one(filter, newvalues)  
print(result.matched_count, result.modified_count )
#adate = datetime(2021,1,13)
#print(dt,adate)
#dt==adate
dt = getMdbDateFromNdays(ndays=10)
filter = {'Date':dt}
for c in db["market_data_close"].find(filter):
    print(c['Date'],c['AAPL'],c['NXGB'])

1 1
2020-12-31 00:00:00 132.69000244140625 0.10509999841451645


### Update

In [34]:
dt = getMdbDateFromNdays(ndays)
#adate = datetime(2021,1,13)
#print(dt,adate)
#dt==adate
filter = {'Date':dt}
#datetime.strptime(strDate,'%Y-%m-%d')
for c in db["market_data_close"].find(filter):
    print(c['Date'],c['AAPL'])
result = market_data.update_one(filter, newvalues)  
print(result.matched_count, result.modified_count )
for c in market_data.find(filter):
    print(c['Date'],c['AAPL'],c['DOG'],c['DOGGY'])

2020-12-31 00:00:00 132.69000244140625


In [ ]:
filter = { 'AAPL': 25 } 
filter = {'Date':datetime(2019, 8, 13, 0, 0)}
newvalues = { "$set": { 'DOG': 'poor boy','DOGGY':'hush now' } } 
  
# Using update_one() method for single  
# updation. 
result = market_data.update_one(filter, newvalues)  
print(result.matched_count, result.modified_count )
for c in market_data.find(filter):
    print(c['Date'],c['AAPL'],c['DOG'],c['DOGGY'])

In [ ]:
adate = datetime(2020,4,20)
filter = {'Date':adate}
db["market_data_close"].find(filter)[0]['ARKG']

In [ ]:
# insert
ndays = 1
dfall = md.getStockPickleNBDays(ndays)
df = dfall['Close']
df.reset_index(inplace=True)
data_dict = df.to_dict("records")
market_data.insert_many(data_dict)
for c in market_data.find():
    print(c['Date'],c['AAPL'])

In [ ]:
import pytz
from tzlocal import get_localzone # $ pip install tzlocal

print(datetime.fromtimestamp(dt / 1e3))
#datetime.fromtimestamp(dt / tzinfo=pytz.utc)
datetime.fromtimestamp(dt/1e3)
utc_dt = datetime(2009, 7, 10, 18, 44, 59, tzinfo=pytz.utc)
print(utc_dt.astimezone(get_localzone())) # print local time
dt
